# PA4 Implementing linear classifiers 
By Group PA4 52 - Zhuangzhuang Gong, Richard Hua


## Task 1

In this predictive system, the "mystery" here is that the second dataset encodes an XOR-style pattern, which no strictly linear classifier can perfectly fit. 

In the first dataset, you can easily draw a single straight-line in the four-dimensional one-hot feature space and separate the "sun" from the others. That's why the perceptron can achieve 100% accuracy with the first dataset.

However, there is a XOR in the second dataset, which means it's not linearly separable. Since both the perceptron and linearSVC can only learn linear decision boundaries, they cannot solve this XOR pattern.

We tried two solutions to solve the problem.

In [1]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline

X1 = [{'city':'Gothenburg', 'month':'July'},
      {'city':'Gothenburg', 'month':'December'},
      {'city':'Paris', 'month':'July'},
      {'city':'Paris', 'month':'December'}]
Y1 = ['rain', 'rain', 'sun', 'rain']

X2 = [{'city':'Sydney', 'month':'July'},
      {'city':'Sydney', 'month':'December'},
      {'city':'Paris', 'month':'July'},
      {'city':'Paris', 'month':'December'}]
Y2 = ['rain', 'sun', 'sun', 'rain']

classifier1 = make_pipeline(DictVectorizer(), Perceptron(max_iter=10))
classifier1.fit(X1, Y1)
guesses1 = classifier1.predict(X1)
print(accuracy_score(Y1, guesses1))

classifier2 = make_pipeline(DictVectorizer(), Perceptron(max_iter=10))
#classifier2 = make_pipeline(DictVectorizer(), LinearSVC())
classifier2.fit(X2, Y2)
guesses2 = classifier2.predict(X2)
print(accuracy_score(Y2, guesses2))

1.0
0.5


**Add an intrerction feature**   
We added an interaction feature to introduce a new feature (i.e., city_month) so that "Sydney_July" becomes its own one-hot. Thus, a linear model on those will fit.

In [4]:
X2_inter = []
for x in X2:
    xi = x.copy()
    xi['city_month'] = f"{x['city']}_{x['month']}"
    X2_inter.append(xi)

print(X2_inter)

[{'city': 'Sydney', 'month': 'July', 'city_month': 'Sydney_July'}, {'city': 'Sydney', 'month': 'December', 'city_month': 'Sydney_December'}, {'city': 'Paris', 'month': 'July', 'city_month': 'Paris_July'}, {'city': 'Paris', 'month': 'December', 'city_month': 'Paris_December'}]


In [5]:
clf = make_pipeline(
    DictVectorizer(),
    Perceptron(max_iter=10)
)
clf.fit(X2_inter, Y2)
print("Accuracy with interaction:", accuracy_score(Y2, clf.predict(X2_inter)))

Accuracy with interaction: 1.0


**Use a non-linear mode**  
Another aspect is to use an RBF kernel to draw a non-linear boundary in the high-dimensional feature space.

RBF can project the data into a high-dimensional space; thus, it can be separated by a curve.

In [6]:
from sklearn.svm import SVC
clf_rbf = make_pipeline(
    DictVectorizer(),
    SVC(kernel='rbf', gamma='scale')   
)
clf_rbf.fit(X2, Y2)
print("RBF SVM accuracy:", accuracy_score(Y2, clf_rbf.predict(X2)))

RBF SVM accuracy: 1.0


## Task 2

### Experiment  
We run the experiment, and the result is around the threshold.

In [7]:
import time
import warnings

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from aml_perceptron import Perceptron, SparsePerceptron

# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels. 
def read_data(corpus_file):
    X = []
    Y = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            _, y, _, x = line.split(maxsplit=3)
            X.append(x.strip())
            Y.append(y)
    return X, Y


if __name__ == '__main__':
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    # Read all the documents.
    X, Y = read_data('data/all_sentiment_shuffled.txt')
    
    # Split into training and test parts.
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                    random_state=0)

    # Set up the preprocessing steps and the classifier.
    pipeline = make_pipeline(
        TfidfVectorizer(),
        SelectKBest(k=1000),
        Normalizer(),

        # NB that this is our Perceptron, not sklearn.linear_model.Perceptron
        Perceptron()  
    )

    # Train the classifier.
    t0 = time.time()
    pipeline.fit(Xtrain, Ytrain)
    t1 = time.time()
    print('Training time: {:.2f} sec.'.format(t1-t0))

    # Evaluate on the test set.
    Yguess = pipeline.predict(Xtest)
    print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))



Training time: 1.20 sec.
Accuracy: 0.7919.


### Task 3

We first checked the consistency of the label and the outlier of the dataset. We didn't find any inconsistency or outlier issues.

In [8]:
from collections import Counter
import pandas as pd
import numpy as np

X, Y = read_data('data/all_sentiment_shuffled.txt')
print(Counter(Y))
Xs = pd.Series(X)
Ys = pd.Series(Y)
print("Outlier X:", Xs.isnull().sum())
print("Outlier Y:",Ys.isnull().sum())

Counter({'pos': 6000, 'neg': 5914})
Outlier X: 0
Outlier Y: 0


**Key steps of the implemention of the algorithm**
1. Label pre-process. Project the two types of labels into +1 / -1 for uniformly process.
2. Weight Initialization. We start a whole-zero vector as the parameter.
3. Training Loop.
    - Repeat for a number of passes. Randomly shuffle the training indices at the start of each pass.
    - For every sample, compute a decaying step size.
    - Shrink the weight vector, if the current sample violates the margin, shift it sightly.
4. Norm projection. Check the weight vector. Rescale it if it grows beyond a shredshold to ensure the model stable.
The sanity check showed that our result is within the expectation.

**Quich takeaways**
- Sweet spot: λ = 1 × 10⁻⁴ gives the best accuracy across all runs.
- Iterations: Raising interation from 10 to 100 gives small gains, but when it comes to 1000, only very slightly accuracy is added to the results while training time multiplies 10 times.

In [9]:
import time
import warnings

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from pegasos import Pegasos
from itertools import product
# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels. 



if __name__ == '__main__':
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    # Read all the documents.
    X, Y = read_data('data/all_sentiment_shuffled.txt')
    
    # Split into training and test parts.
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                    random_state=0)
    
    param_space = {
    "n_iter":        [10, 20, 100, 200],
    "lambda_param":  [1e-5, 1e-4, 1e-3],
    }

    results = []
    # Iterate over all combinations of parameters.
    # We use the product function from itertools to create a cartesian
    for n_iter, lam in product(param_space["n_iter"], param_space["lambda_param"]):
        # Set up the preprocessing steps and the classifier.
        clf = make_pipeline(
            TfidfVectorizer(),
            SelectKBest(k=1000),
            Normalizer(),
            # NB that this is our Pegasos. See the implementation in the according .py file
            Pegasos(n_iter=n_iter, lambda_param=lam)
        )

        t0 = time.time()
        clf.fit(Xtrain, Ytrain)
        train_time = time.time() - t0
        acc = accuracy_score(Ytest, clf.predict(Xtest)) 

        results.append(
            {"n_iter": n_iter,
            "lambda": lam,
            "train_time(s)": round(train_time, 2),
            "test_acc": round(acc, 4)}
        )
        print(f"n_iter={n_iter:<3} λ={lam:<8} | time {train_time:5.1f}s | acc {acc:.4f}")
    # Sort the results by accuracy.
    df = pd.DataFrame(results).sort_values("test_acc", ascending=False)
    print("\n=== Result Conclusion ===")
    print(df.to_string(index=False))

n_iter=10  λ=1e-05    | time   2.1s | acc 0.8233
n_iter=10  λ=0.0001   | time   2.3s | acc 0.8359
n_iter=10  λ=0.001    | time   2.3s | acc 0.8254
n_iter=20  λ=1e-05    | time   4.7s | acc 0.8355
n_iter=20  λ=0.0001   | time   4.2s | acc 0.8292
n_iter=20  λ=0.001    | time   4.8s | acc 0.8254
n_iter=100 λ=1e-05    | time  16.1s | acc 0.8342
n_iter=100 λ=0.0001   | time  13.5s | acc 0.8363
n_iter=100 λ=0.001    | time  15.3s | acc 0.8254
n_iter=200 λ=1e-05    | time  21.9s | acc 0.8334
n_iter=200 λ=0.0001   | time  22.5s | acc 0.8372
n_iter=200 λ=0.001    | time  24.3s | acc 0.8267

=== Result Conclusion ===
 n_iter  lambda  train_time(s)  test_acc
    200 0.00010          22.49    0.8372
    100 0.00010          13.55    0.8363
     10 0.00010           2.33    0.8359
     20 0.00001           4.68    0.8355
    100 0.00001          16.14    0.8342
    200 0.00001          21.94    0.8334
     20 0.00010           4.17    0.8292
    200 0.00100          24.29    0.8267
     10 0.00100 

## Task 4

In thie section, we compared the algprithms, using the log loss instead of the hinge loss.

**Key Takeaways**  
- Accuracy: both peak around the same level. The accuracy gap is tiny.  
- Training time: hinge loss updates only on “violators”, so it reaches its plateau faster. Log loss always updates, so costs ~30 % more wall-time for the same number of passes.  
- λ-sensitivity: both deteriorate when λ is too big or too small, but log loss drops harder at λ = 0.001 whereas hinge loss stays ~0.826.

In [10]:
import time
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from itertools import product
from pegasos import LogisticRegression

# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels. 
def read_data(corpus_file):
    X = []
    Y = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            _, y, _, x = line.split(maxsplit=3)
            X.append(x.strip())
            Y.append(y)
    return X, Y


if __name__ == '__main__':
    warnings.filterwarnings("ignore", category=FutureWarning)

    # Read all the documents.
    X, Y = read_data('data/all_sentiment_shuffled.txt')
    
    # Split into training and test parts.
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                    random_state=0)

    # ---------- parameter grid ----------
    grid_n_iter   = [10, 20, 100, 200]          # epochs
    grid_lambda   = [1e-3, 1e-4, 1e-5]     # regulariser
    results = []

    # ---------- loop ----------
    for n_iter, lam in product(grid_n_iter, grid_lambda):
        clf = make_pipeline(
            TfidfVectorizer(),
            SelectKBest(k=1000),
            Normalizer(),
            LogisticRegression(n_iter=n_iter, lambda_param=lam)
        )

        t0 = time.time()
        clf.fit(Xtrain, Ytrain)
        train_time = time.time() - t0
        acc = accuracy_score(Ytest, clf.predict(Xtest))

        results.append(
            {"n_iter": n_iter,
            "lambda": lam,
            "train_time(s)": round(train_time, 2),
            "test_acc": round(acc, 4)}
        )
        print(f"n_iter={n_iter:<4} λ={lam:<.0e}  |  time {train_time:5.2f}s  |  acc {acc:.4f}")

    # ---------- summary ----------
    df = pd.DataFrame(results).sort_values("test_acc", ascending=False)
    print("\n=== Result Conclusion ===")
    print(df.to_string(index=False))


n_iter=10   λ=1e-03  |  time  3.73s  |  acc 0.8065
n_iter=10   λ=1e-04  |  time  2.30s  |  acc 0.8321


D:\code\AML\Applied-Machine-Learning-Lab\PA4\pegasos.py:146: RuntimeWarning: overflow encountered in exp
  loss = self.lambda_param * self.w - (y * x) / (1 + np.exp(part))


n_iter=10   λ=1e-05  |  time  1.99s  |  acc 0.8225
n_iter=20   λ=1e-03  |  time  3.13s  |  acc 0.8082
n_iter=20   λ=1e-04  |  time  3.51s  |  acc 0.8330


D:\code\AML\Applied-Machine-Learning-Lab\PA4\pegasos.py:146: RuntimeWarning: overflow encountered in exp
  loss = self.lambda_param * self.w - (y * x) / (1 + np.exp(part))


n_iter=20   λ=1e-05  |  time  3.00s  |  acc 0.8284
n_iter=100  λ=1e-03  |  time 11.15s  |  acc 0.8074
n_iter=100  λ=1e-04  |  time 12.75s  |  acc 0.8376


D:\code\AML\Applied-Machine-Learning-Lab\PA4\pegasos.py:146: RuntimeWarning: overflow encountered in exp
  loss = self.lambda_param * self.w - (y * x) / (1 + np.exp(part))


n_iter=100  λ=1e-05  |  time 42.78s  |  acc 0.8313
n_iter=200  λ=1e-03  |  time 46.08s  |  acc 0.8074
n_iter=200  λ=1e-04  |  time 33.86s  |  acc 0.8384


D:\code\AML\Applied-Machine-Learning-Lab\PA4\pegasos.py:146: RuntimeWarning: overflow encountered in exp
  loss = self.lambda_param * self.w - (y * x) / (1 + np.exp(part))


n_iter=200  λ=1e-05  |  time 41.57s  |  acc 0.8326

=== Result Conclusion ===
 n_iter  lambda  train_time(s)  test_acc
    200 0.00010          33.86    0.8384
    100 0.00010          12.75    0.8376
     20 0.00010           3.51    0.8330
    200 0.00001          41.57    0.8326
     10 0.00010           2.30    0.8321
    100 0.00001          42.78    0.8313
     20 0.00001           3.00    0.8284
     10 0.00001           1.99    0.8225
     20 0.00100           3.13    0.8082
    100 0.00100          11.15    0.8074
    200 0.00100          46.08    0.8074
     10 0.00100           3.73    0.8065


## Bonus task 1


### A. Faster linear algebra operations
We used  ddot(x, y) to repalce x.dot(y) adn dscal to replace '*=' in our original implementation of the pegasos algorithm.

We evaluated three parameter settings using iteration counts of 10, 100, and 200 to demonstrate the optimized algorithm’s performance under different conditions comparing to the original algorithm.

For time consumption aspect, performance under all three parameter settings' has been improved.
Training time sees substantial savings at higher iteraton counts. Noticeably, the time almost reduced by 46% at 200 iterations.

In [11]:
import time
import warnings

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from pegasos import Pegasos_faster_linear
from itertools import product
# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels. 



if __name__ == '__main__':
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    # Read all the documents.
    X, Y = read_data('data/all_sentiment_shuffled.txt')
    
    # Split into training and test parts.
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                    random_state=0)
    
    param_space = {
    "n_iter":        [20, 100, 200],
    "lambda_param":  [ 1e-4],
    }

    results = []
    # Iterate over all combinations of parameters.
    # We use the product function from itertools to create a cartesian
    for n_iter, lam in product(param_space["n_iter"], param_space["lambda_param"]):
        # Set up the preprocessing steps and the classifier.
        clf = make_pipeline(
            TfidfVectorizer(),
            SelectKBest(k=1000),
            Normalizer(),
            # NB that this is our Pegasos. See the implementation in the according .py file
            Pegasos_faster_linear(n_iter=n_iter, lambda_param=lam)
        )

        t0 = time.time()
        clf.fit(Xtrain, Ytrain)
        train_time = time.time() - t0
        acc = accuracy_score(Ytest, clf.predict(Xtest)) 

        results.append(
            {"n_iter": n_iter,
            "lambda": lam,
            "train_time(s)": round(train_time, 2),
            "test_acc": round(acc, 4)}
        )
        print(f"n_iter={n_iter:<3} λ={lam:<8} | time {train_time:5.1f}s | acc {acc:.4f}")
    # Sort the results by accuracy.
    df = pd.DataFrame(results).sort_values("test_acc", ascending=False)
    print("\n=== Result Conclusion ===")
    print(df.to_string(index=False))

n_iter=20  λ=0.0001   | time   2.9s | acc 0.8342
n_iter=100 λ=0.0001   | time  11.6s | acc 0.8389
n_iter=200 λ=0.0001   | time  19.2s | acc 0.8401

=== Result Conclusion ===
 n_iter  lambda  train_time(s)  test_acc
    200  0.0001          19.17    0.8401
    100  0.0001          11.62    0.8389
     20  0.0001           2.92    0.8342


### B. Using sparse vectors

**Remove the SelectKBest step**  

We removed the SelectKBest from the steps to better explore the optimization. It turns out that after removing the SelectKBest step, the time comsumption increased almost 20 times. 

One reason behind this is that the SelectKbest can reduce the dimensionality which can shrink the scale of the computing. Another reason is that this function can filter the low weight words, saving the cost of total computing and also reduce the risk of overfitting.

In [12]:
import time
import warnings

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from pegasos import Pegasos
from itertools import product
# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels. 



if __name__ == '__main__':
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    # Read all the documents.
    X, Y = read_data('data/all_sentiment_shuffled.txt')
    
    # Split into training and test parts.
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                    random_state=0)
    
    param_space = {
    "n_iter":        [100],
    "lambda_param":  [1e-4],
    }

    results = []
    # Iterate over all combinations of parameters.
    # We use the product function from itertools to create a cartesian
    for n_iter, lam in product(param_space["n_iter"], param_space["lambda_param"]):
        # Set up the preprocessing steps and the classifier.
        clf = make_pipeline(
            TfidfVectorizer(),
            #SelectKBest(k=1000),
            Normalizer(),
            # NB that this is our Pegasos. See the implementation in the according .py file
            Pegasos(n_iter=n_iter, lambda_param=lam)
        )
    
        t0 = time.time()
        clf.fit(Xtrain, Ytrain)
        train_time = time.time() - t0
        acc = accuracy_score(Ytest, clf.predict(Xtest)) 

        results.append(
            {"n_iter": n_iter,
            "lambda": lam,
            "train_time(s)": round(train_time, 2),
            "test_acc": round(acc, 4)}
        )
        print(f"n_iter={n_iter:<3} λ={lam:<8} | time {train_time:5.1f}s | acc {acc:.4f}")
    # Sort the results by accuracy.
    df = pd.DataFrame(results).sort_values("test_acc", ascending=False)
    print("\n=== Result Conclusion ===")
    print(df.to_string(index=False))

n_iter=100 λ=0.0001   | time 287.9s | acc 0.8418

=== Result Conclusion ===
 n_iter  lambda  train_time(s)  test_acc
    100  0.0001         287.94    0.8418


 **Include Two-word Sequences**  

 We then tried adding the option ngram_range=(1,2) to the TfidfVectorizer. 
 
 It includes the bigram as a feature as well, which richers the feature set but also increase the dimensionality, meaning more memory and slower operations.

 This exploration needs too much time to run, thus we don't exhibit our result here. However, through the expriment, we understand the trade off between counts of feature and effeciecncy.

In [ ]:
# import time
# import warnings

# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.preprocessing import Normalizer
# from sklearn.pipeline import make_pipeline
# from sklearn.feature_selection import SelectKBest
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split

# from pegasos import Pegasos
# from itertools import product
# # This function reads the corpus, returns a list of documents, and a list
# # of their corresponding polarity labels. 



# if __name__ == '__main__':
#     warnings.filterwarnings("ignore", category=FutureWarning)
    
#     # Read all the documents.
#     X, Y = read_data('data/all_sentiment_shuffled.txt')
    
#     # Split into training and test parts.
#     Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
#                                                     random_state=0)
    
#     param_space = {
#     "n_iter":        [100],
#     "lambda_param":  [1e-4],
#     }

#     results = []
#     # Iterate over all combinations of parameters.
#     # We use the product function from itertools to create a cartesian
#     for n_iter, lam in product(param_space["n_iter"], param_space["lambda_param"]):
#         # Set up the preprocessing steps and the classifier.
#         clf = make_pipeline(
#             TfidfVectorizer(ngram_range=(1,2)),
#             #SelectKBest(k=1000),
#             Normalizer(),
#             # NB that this is our Pegasos. See the implementation in the according .py file
#             Pegasos(n_iter=n_iter, lambda_param=lam)
#         )
    
#         t0 = time.time()
#         clf.fit(Xtrain, Ytrain)
#         train_time = time.time() - t0
#         acc = accuracy_score(Ytest, clf.predict(Xtest)) 

#         results.append(
#             {"n_iter": n_iter,
#             "lambda": lam,
#             "train_time(s)": round(train_time, 2),
#             "test_acc": round(acc, 4)}
#         )
#         print(f"n_iter={n_iter:<3} λ={lam:<8} | time {train_time:5.1f}s | acc {acc:.4f}")
#     # Sort the results by accuracy.
#     df = pd.DataFrame(results).sort_values("test_acc", ascending=False)
#     print("\n=== Result Conclusion ===")
#     print(df.to_string(index=False))

**Using sparse vectors**  
By switching to sparse vector representations, we kept the same hyperparameters (e.g. n_iter, λ) and achieved nearly identical test accuracy (≈83.9%) while greatly reducing memory pressure. 

However, because weight decay and norm projection still run over the full feature dimension (O(d)), training time actually rose when d reverted to its original size. This shows that sparse updates alone relieve memory constraints but that additional feature reduction or mini-batch updates are required to maintain fast training.

In [13]:
import time
import warnings

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from pegasos import Pegasos_sparse_vectors
from itertools import product
# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels. 



if __name__ == '__main__':
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    # Read all the documents.
    X, Y = read_data('data/all_sentiment_shuffled.txt')
    
    # Split into training and test parts.
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                    random_state=0)
    
    param_space = {
    "n_iter":        [10, 20, 100, 200],
    "lambda_param":  [1e-4],
    }

    results = []
    # Iterate over all combinations of parameters.
    # We use the product function from itertools to create a cartesian
    for n_iter, lam in product(param_space["n_iter"], param_space["lambda_param"]):
        # Set up the preprocessing steps and the classifier.
        clf = make_pipeline(
            TfidfVectorizer(),
            SelectKBest(k=1000),
            Normalizer(),
            # NB that this is our Pegasos. See the implementation in the according .py file
            Pegasos_sparse_vectors(n_iter=n_iter, lambda_param=lam)
        )

        t0 = time.time()
        clf.fit(Xtrain, Ytrain)
        train_time = time.time() - t0
        acc = accuracy_score(Ytest, clf.predict(Xtest)) 

        results.append(
            {"n_iter": n_iter,
            "lambda": lam,
            "train_time(s)": round(train_time, 2),
            "test_acc": round(acc, 4)}
        )
        print(f"n_iter={n_iter:<3} λ={lam:<8} | time {train_time:5.1f}s | acc {acc:.4f}")
    # Sort the results by accuracy.
    df = pd.DataFrame(results).sort_values("test_acc", ascending=False)
    print("\n=== Result Conclusion ===")
    print(df.to_string(index=False))

n_iter=10  λ=0.0001   | time  90.8s | acc 0.8410
n_iter=20  λ=0.0001   | time  34.2s | acc 0.8376
n_iter=100 λ=0.0001   | time 141.7s | acc 0.8359
n_iter=200 λ=0.0001   | time 230.3s | acc 0.8363

=== Result Conclusion ===
 n_iter  lambda  train_time(s)  test_acc
     10  0.0001          90.83    0.8410
     20  0.0001          34.23    0.8376
    200  0.0001         230.31    0.8363
    100  0.0001         141.73    0.8359


### C. Speeding up the scaling operation

In [14]:
import time
import warnings

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from pegasos import Pegasos_vec_scale
from itertools import product
# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels. 



if __name__ == '__main__':
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    # Read all the documents.
    X, Y = read_data('data/all_sentiment_shuffled.txt')
    
    # Split into training and test parts.
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                    random_state=0)
    
    param_space = {
    "n_iter":        [10, 100, 200],
    "lambda_param":  [ 1e-4],
    }

    results = []
    # Iterate over all combinations of parameters.
    # We use the product function from itertools to create a cartesian
    for n_iter, lam in product(param_space["n_iter"], param_space["lambda_param"]):
        # Set up the preprocessing steps and the classifier.
        clf = make_pipeline(
            TfidfVectorizer(),
            SelectKBest(k=1000),
            Normalizer(),
            # NB that this is our Pegasos. See the implementation in the according .py file
            Pegasos_vec_scale(n_iter=n_iter, lambda_param=lam)
        )

        t0 = time.time()
        clf.fit(Xtrain, Ytrain)
        train_time = time.time() - t0
        acc = accuracy_score(Ytest, clf.predict(Xtest)) 

        results.append(
            {"n_iter": n_iter,
            "lambda": lam,
            "train_time(s)": round(train_time, 2),
            "test_acc": round(acc, 4)}
        )
        print(f"n_iter={n_iter:<3} λ={lam:<8} | time {train_time:5.1f}s | acc {acc:.4f}")
    # Sort the results by accuracy.
    df = pd.DataFrame(results).sort_values("test_acc", ascending=False)
    print("\n=== Result Conclusion ===")
    print(df.to_string(index=False))

n_iter=10  λ=0.0001   | time   1.4s | acc 0.8338
n_iter=100 λ=0.0001   | time   5.9s | acc 0.8368
n_iter=200 λ=0.0001   | time  10.0s | acc 0.8368

=== Result Conclusion ===
 n_iter  lambda  train_time(s)  test_acc
    100  0.0001           5.86    0.8368
    200  0.0001           9.98    0.8368
     10  0.0001           1.41    0.8338


We sped up the scaling operations by extending our solution from part a) of the bonus task. We minimized the number of vector scaling operations by defining a constant alpha which is scaled at each iteration instead of the vector. This makes a difference when the vector is especially long. An issue with this approach is that the constant, alpha, tends to get very, very small. In our testing, this led to arithmetic errors as alpha approached zero. To remedy this, we decided to fix a minimum value for alpha at 10^-8. This was accomplished by always taking the maximum of either alpha or 10^-8 at each iteration. After some further testing, we found that fixing a minimum value for alpah did not significantly impact the accuracy. The accuracy scores were comparable to the scores from a). In our results we can see that the run times have overall decreased around 20% from part a). 